In [ ]:
import sys, os
sys.path.append("/b2p")
BASE_DIR = os.getcwd()

In [117]:
import osmnx as ox
import geopandas as gpd
import rasterio
from rasterio import features
import shapely
from src.utilities.coords import *
from src.utilities.imaging import *
from pyproj import CRS


## Tags
I obtained the tags by looking through this [documentation](https://wiki.openstreetmap.org/wiki/Map_features)

In [95]:
tags_water = {
    'water': True,
    'waterway': True
}
tags_roads = {
    'Highway': True
}
tags_pop = {
    'Place': True # subtypes are 
        # - Administratively declared places
        # - Populated settlements, urban
        # - Populated settlements, urban and rural
        # - Other places
}
tags_natural = {
    'Natural': True # subtypes are 
        # - Administratively declared places
        # - Populated settlements, urban
        # - Populated settlements, urban and rural
        # - Other places
}

## Test rasterization
The idea is to take a composite from S2 and then get corresponding water, roads, and population information  

In [98]:
# Rwanda 35MPS 
s2_tiff = '/Users/user/Desktop/35MPS_multiband.tiff'
out_tiff = '/Users/user/Desktop/35MPS_multivariate.tiff'
(tl,tr,br,bl) = tiff_to_bbox(s2_tiff)
bbox = [tl[0], br[0], tl[1], br[1]] # convert to (N,S,E,W)
print(*bbox)

-2.713837411553505 -3.7055145428311884 27.899639974853475 28.88900139325669


In [96]:
water = ox.geometries.geometries_from_bbox(*bbox, tags_water)
roads = ox.geometries.geometries_from_bbox(*bbox, tags_roads)
pop = ox.geometries.geometries_from_bbox(*bbox, tags_pop)
natural = ox.geometries.geometries_from_bbox(*bbox, tags_natural)

In [63]:
def latLonToTiffTform(tiff):
    src = gdal.Open(tiff)
    # Setup the source projection - you can also import from epsg, proj4...
    srcRef = osr.SpatialReference()
    srcRef.ImportFromWkt(src.GetProjection())

    # The target projection
    tgtRef = osr.SpatialReference()
    tgtRef.ImportFromEPSG(4326) # this code gives us lat, long

    # Create the transform - this can be used repeatedly
    return osr.CoordinateTransformation(tgtRef,srcRef)

In [138]:
with rasterio.open(s2_tiff, 'r') as rst:
    # copy and update the metadata from the input raster for the output
    waterways = water.loc[:,['geometry', 'waterway']].dropna()
    unique_vals = waterways.waterway.unique()
    meta = rst.meta.copy()
    d = meta['count']
    meta.update(
        compress='lzw',
        count=d+1#len(unique_vals)
    )
    waterways = waterways.to_crs('epsg:32735')
    # tform = latLonToTiffTform(s2_tiff)
    with rasterio.open(out_tiff, 'w+', **meta) as out:
        print('writing previous raster data to multivariate file... ')
        for i in range(d): 
            out.write_band(i+1, rst.read(i+1))
        print('getting list of polygons that define our water features ')
        # this is where we create a generator of geom, value pairs to use in rasterizing
        out_arr = out.read(d+1)
        shapes = ((geom,1) for j,geom in enumerate(waterways['geometry']) )
        # Now burn the features into the raster and write it out
        print('writing water to raster ')
        water_arr = features.rasterize(
            shapes=shapes, 
            fill=0, 
            out=out_arr, 
            transform=out.transform #tform
        )
        assert (water_arr > 0).any(), 'Water array is empty'
        out.write_band(d+1, water_arr)


## Test function definition

In [ ]:
from src.osm import getOsm

s2_tiff = '/Users/user/Desktop/35MPS_multiband.tiff'
out_tiff = '/Users/user/Desktop/35MPS_multivariate_test.tiff'
getOSM(s2_tiff, out_tiff)